<a href="https://colab.research.google.com/github/soufianeodf/Quora-Insincere-Questions-Classification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# this is a test
# this is a second line